In [1]:
import theseus as th
import numpy as np

#for checking computation times (time.perf_counter())
import time

In [1]:
#define parameters of system
pdv =(5,3,8)

#construct GHZ state from (p,d,v)
def makeGHZ(pdv):
    state = []
    data, dim, verts = pdv
    for ii in range(dim):
        term = []
        for jj in range(verts):
            if jj<data:
                term.append((jj,ii))
            else:
                term.append((jj,0))
        state.append(term)
    return state
        
ghz = makeGHZ(pdv)

In [3]:
ghz

[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 0), (6, 0), (7, 0)],
 [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 0), (6, 0), (7, 0)]]

In [4]:
#build starting graph for system and find all perfect matchings 
t0=time.perf_counter()
locdim = [pdv[1]]*pdv[0]+[1]*(pdv[2]-pdv[0])
edge_list = th.buildAllEdges(locdim)
pms = th.findEdgeCovers(edge_list,order = 0)
print(time.perf_counter()-t0)

8.912102286


In [5]:
#function for simplifying edge_list if wanted
def makeUnicolor(edge_list):
    return [edge for edge in edge_list if (((edge[0] not in range(5)) or (edge[1] not in range(5))) or (edge[2]==edge[3]))]

In [6]:
edge_list = makeUnicolor(edge_list)

In [7]:
#key numbers about the system

print('number of edges in starting graph:')
print(len(edge_list))
print('\n')

print('perfect matchings of uncolored graph')
arbitrarycoloring = np.array(pms[0])[:,2:4].flatten()
print(len([pm for pm in pms if np.array_equal(np.array(pm)[:,2:4].flatten(),arbitrarycoloring)]))
print('\n')

print('terms in normalization constant')
statedict = th.allEdgeCovers(locdim,order=0)
print(len(statedict))

number of edges in starting graph:
78


perfect matchings of uncolored graph
105


terms in normalization constant
243


In [8]:
t0 = time.perf_counter()
#define important functions
target = th.targetEquation([1]*pdv[1],ghz)
norm = th.Norm.fromEdgeCovers(edge_list)
print(time.perf_counter()-t0)

5.9188233049999965


In [9]:
#define loss
sc_loss = 1 - target/(1+norm)
fid_loss = 1 - target/norm

loss = sc_loss

## functions

In [35]:
#function for manually checking the values
def orderVals(vals):
    vallist = list(vals.values())
    vallist = [[x,i] for i,x in enumerate(vallist)]
    vallist.sort(key=lambda s: abs(s[0]),reverse=True)
    x_reordered, inds = np.array(vallist)[:,0], np.array(vallist)[:,1].astype(int)
    keylist = list(vals.keys())
    variables_reordered = [keylist[i] for i in inds]
    vals_reordered = dict(np.transpose([variables_reordered,x_reordered]))
    return vals_reordered

In [11]:
def variablesFromLoss(loss):
    variables = list(loss.free_symbols)
    variables.sort(key=lambda s: str(s))
    return variables

In [27]:
def subsLoss(function, sol, zero_vals=dict()):
    simple_loss = function.subs(zero_vals)
    variables = variablesFromLoss(simple_loss)
    return simple_loss.subs(dict(np.transpose([variables,sol.x])))

# Optimization 

In [76]:
t0 = time.perf_counter()
#initial optimization with all weights
#variables = list(loss.free_symbols)
#variables.sort(key=lambda s: str(s))
variables = variablesFromLoss(loss)
bounds = len(variables)*[(-1,1)]

cont = True
while cont:
    print('optimizing')
    sol = th.sympyMinimizer(loss,variables = variables,bounds=bounds,method = 'L-BFGS-B',options={'disp':99,'maxfun': 15000})
    if sol.fun<0.1: cont = False
print(time.perf_counter()-t0)

optimizing
optimizing
optimizing
68.99820227800228


In [77]:
t0 = time.perf_counter()
#dictionary associating variables with values
vals_start = dict(np.transpose([variables,sol.x]))

#dictionary keeping track of deleted edges
zero_vals_start = dict()

#delete edges below a threshold
thr_initial = 0.000001
for variable in variables:
    if abs(vals_start[variable]) < thr_initial:
        #vals_start[variable]= 0
        zero_vals_start[variable] = 0
        
print('edges deleted in first optimization:')
print(len(zero_vals_start))
        
# set some variables to zero with dict zero_vals_start
loss_start = loss.subs(zero_vals_start)
print('defined new loss')

fid_start = fid_loss.subs(zero_vals_start)
print('defined new fid loss')

variables_start = variablesFromLoss(loss_start)

#make new optimization with truncated graph
bounds_start = len(variables_start)*[(-1,1)]
sol_start = th.sympyMinimizer(loss_start,variables = variables_start,bounds=bounds_start,method = 'L-BFGS-B',options={'disp':99,'maxfun': 15000})
print('found new solution')
print('time')
print(time.perf_counter()-t0)

edges deleted in first optimization:
3
defined new loss
defined new fid loss
found new solution
time
23.320909941001446


## topopt

In [97]:
#EVALUATE TO RESTART FROM FIRST TRUNCATED SOLUTION

loss_cur = loss_start
fid_cur = fid_start
variables_cur = variables_start
sol_cur = sol_start
vals_cur = dict(np.transpose([variables_cur,sol_cur.x]))
zero_vals_cur = zero_vals_start

In [98]:
t0=time.perf_counter()
success = False
rep = 0
rep2 = 0
opt_time = []

while rep <10:
    print('--------')
    print('rep: '+str(rep)+'    '+'rep2: '+str(rep2)+'    '+str(len(zero_vals_cur)))
    #vals is dictionary only for the variables actually contained in loss_cur
    vals = vals_cur.copy()
    
    #dictionary of zero weights, ready to add new entries
    zero_vals_new = zero_vals_cur.copy()
    
    
        #delete rep'th smallest weight
    vals_ordered = orderVals(vals)
    #print(vals_ordered)
    variables_ordered = list(vals_ordered.keys())
    del_var = variables_ordered[-1-rep]
    print('deleting just one edge '+str(del_var)+' '+str(vals[del_var]))
    zero_vals_new[del_var] = 0
        
    if rep2 ==0: #TRYING NEW EDGE
        loss_new = loss_cur.subs(zero_vals_new)
        variables_new = variablesFromLoss(loss_new)
        bounds_new = len(variables_new)*[(-1,1)]
        vals_new = vals.copy()
        vals_new.pop(del_var,None)
        init_vals = list(vals_new.values())
        
    #FINDING NEW SOLUTION
    t1=time.perf_counter()
    if rep2 <=1: #try values of previous iteration
        sol_new = th.sympyMinimizer(loss_new,variables = variables_new,initial_values = init_vals,bounds=bounds_new,method = 'SLSQP',options={'disp':99})
    else: #try random initial values
        sol_new = th.sympyMinimizer(loss_new,variables = variables_new,bounds=bounds_new,method = 'SLSQP',options={'disp':99})
    opt_t = time.perf_counter()-t1
    opt_time.append(opt_t)
    print('optimizer took '+str(opt_t)+' seconds')
    #https://mathematica.stackexchange.com/questions/181266/findminimum-how-can-i-know-which-method-mathematica-has-used
    #https://reference.wolfram.com/language/tutorial/ConstrainedOptimizationLocalNumerical.html
    
    #CHECKING IF NEW SOLUTION IS GOOD      
    if sol_new.fun<0.1: #CHECKING COUNT RATE
        fid_new = fid_cur.subs(zero_vals_new) #simplifying fidelity
        fid_check = subsLoss(fid_new,sol_new)
        if fid_check < 0.05: #CHECKING FIDELITY
            print('************ GOOD NEW SOL FOUND, updating for next iteration ************')
            print('edges left:'+str(len(variables_new)))
            success = True
            zero_vals_cur = zero_vals_new.copy()
            loss_cur = loss_new
            fid_cur = fid_new
            variables_cur = variables_new #list of varibale names
            sol_cur = sol_new #variables_new was used to generate sol_new --> they have the same order
            vals_cur = dict(np.transpose([variables_cur,sol_cur.x])) #dictionary  dict[variable name]=weight
            rep = 0
            rep2 = 0
        else:
            print('new sol wasnt good.....fid '+str(fid_check))
            rep2 += 1
            if rep2 >=5:
                rep += 1
                rep2 =0
            
    else:
        print('new sol wasnt good.....cr '+str(sol_new.fun))
        rep2 += 1
        if rep2 >=5:
            rep += 1
            rep2 =0

print('edges left:')
print(len(variables_cur))
if success: 
    print(vals_cur)
    print('fidelity')
    print(1-fid_cur.subs(vals_cur))
print('time')
total_time = time.perf_counter()-t0
print(total_time)

--------
rep: 0    rep2: 0    3
deleting just one edge w_00,04^1,1 -5.672891973737485e-06
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.03483309021126957
            Iterations: 3
            Function evaluations: 225
            Gradient evaluations: 3
optimizer took 14.408128046001366 seconds
************ GOOD NEW SOL FOUND, updating for next iteration ************
edges left:74
--------
rep: 0    rep2: 0    4
deleting just one edge w_00,02^1,1 1.547573849177431e-06
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.034832644867389684
            Iterations: 1
            Function evaluations: 75
            Gradient evaluations: 1
optimizer took 12.850324065999303 seconds
************ GOOD NEW SOL FOUND, updating for next iteration ************
edges left:73
--------
rep: 0    rep2: 0    5
deleting just one edge w_00,04^2,2 -1.1954862441416352e-06
Optimization terminated successfully    (Exit mode